In [164]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import imageio
import numpy as np
import warnings
import torch
from torch import nn
import torch.optim as opt

In [249]:
tracking1=pd.read_csv('tracking_week_1.csv')
plays=pd.read_csv('plays.csv')

tracking1.loc[tracking1['playDirection'] == 'left', 'x'] = 120 - tracking1.loc[tracking1['playDirection'] == 'left', 'x']
tracking1.loc[tracking1['playDirection'] == 'left', 'y'] = (160/3) - tracking1.loc[tracking1['playDirection'] == 'left', 'y']
tracking1.loc[tracking1['playDirection'] == 'left', 'dir'] += 180
tracking1.loc[tracking1['dir'] > 360, 'dir'] -= 360
tracking1.loc[tracking1['playDirection'] == 'left', 'o'] += 180
tracking1.loc[tracking1['o'] > 360, 'o'] -= 360

tracking1_with_plays = tracking1.merge(plays, on=['gameId', 'playId'], how='left')
tracking1_with_plays = tracking1_with_plays[tracking1_with_plays['playNullifiedByPenalty'] == 'N']
tracking1_with_plays['is_on_offence'] = tracking1_with_plays['club'] == tracking1_with_plays['possessionTeam']
tracking1_with_plays['is_on_defence'] = tracking1_with_plays['club'] == tracking1_with_plays['defensiveTeam']
tracking1_with_plays['is_ballcarrier'] = tracking1_with_plays['ballCarrierId'] == tracking1_with_plays['nflId']

tracking1_with_plays

,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,...,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2,is_on_offence,is_on_defence,is_ballcarrier
0,2022090800,56,35472.0,Rodger Saffold,1,2022-09-08 20:24:05.200000,76.0,BUF,left,31.630000,...,0.000031,1.298699,0.004420,NaN,NaN,NaN,NaN,True,False,False
1,2022090800,56,35472.0,Rodger Saffold,2,2022-09-08 20:24:05.299999,76.0,BUF,left,31.530000,...,0.000031,1.298699,0.004420,NaN,NaN,NaN,NaN,True,False,False
2,2022090800,56,35472.0,Rodger Saffold,3,2022-09-08 20:24:05.400000,76.0,BUF,left,31.440000,...,0.000031,1.298699,0.004420,NaN,NaN,NaN,NaN,True,False,False
3,2022090800,56,35472.0,Rodger Saffold,4,2022-09-08 20:24:05.500000,76.0,BUF,left,31.360000,...,0.000031,1.298699,0.004420,NaN,NaN,NaN,NaN,True,False,False
4,2022090800,56,35472.0,Rodger Saffold,5,2022-09-08 20:24:05.599999,76.0,BUF,left,31.280000,...,0.000031,1.298699,0.004420,NaN,NaN,NaN,NaN,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,NaN,football,49,2022-09-12 23:05:57.799999,NaN,football,left,63.779999,...,-0.282255,0.967420,-0.719924,NaN,NaN,NaN,NaN,False,False,False
1407435,2022091200,3826,NaN,football,50,2022-09-12 23:05:57.900000,NaN,football,left,63.939999,...,-0.282255,0.967420,-0.719924,NaN,NaN,NaN,NaN,False,False,False
1407436,2022091200,3826,NaN,football,51,2022-09-12 23:05:58.000000,NaN,football,left,64.110001,...,-0.282255,0.967420,-0.719924,NaN,NaN,NaN,NaN,False,False,False
1407437,2022091200,3826,NaN,football,52,2022-09-12 23:05:58.099999,NaN,football,left,64.270000,...,-0.282255,0.967420,-0.719924,NaN,NaN,NaN,NaN,False,False,False


In [55]:
tracking1_with_plays['o']

0          51.74
1          50.98
2          50.98
3          52.38
4          53.36
           ...  
1407434      NaN
1407435      NaN
1407436      NaN
1407437      NaN
1407438      NaN
Name: o, Length: 1370386, dtype: float64

In [250]:
def preprocessing(df, past=False):
    df['radiansDirection'] = df['dir'].astype(float).apply(math.radians) #Converts angle in degrees to radians
    df['xComponent']=df['radiansDirection'].astype(float).apply(math.cos) #Converts angle into an x and y component
    df['yComponent']=df['radiansDirection'].astype(float).apply(math.sin)
    df['xspeed']=df['xComponent']*df['s'] #Determines magnitude of speed by multiplying x and y component by magnitude of speed
    df['yspeed']=df['yComponent']*df['s']
    if past:
        df['OLD_x']=df['x']
        df['OLD_y']=df['y']
        df['OLD_xspeed']=df['xspeed']
        df['OLD_yspeed']=df['yspeed']
        df['OLD_o']=df['o']
        df['OLD_s']=df['s']
        df['OLD_dir']=df['dir']
        df['PROJ_x']=df['OLD_x']+df['OLD_xspeed']*0.1
        df['PROJ_y']=df['OLD_y']+df['OLD_yspeed']*0.1
        return df[['gameId', 'playId', 'nflId', 'OLD_x', 'OLD_y', 'OLD_s', 'OLD_o', 'OLD_dir', 'OLD_xspeed', 'OLD_yspeed', 'PROJ_x', 'PROJ_y']]
    return df

In [118]:
def model(df, nflId):
    if nflId==38577:
        return 1
    return 0

In [288]:
def reformat(frame):
    #Initilize ball carrier row
    ballcarrier = 0
    #Create empty array
    temp = torch.zeros(8,22,22)
    #Get vector of defence and offence teams
    
    for i in range(len(frame['x'])):
        #Assign ballcarrier if player is ballcarrier
        if frame.iloc[i]["is_ballcarrier"]:
            ballcarrier = i
        #Assign team vector:
        if frame.iloc[i]['is_on_offence']:
            teamvec2 = frame['is_on_offence']
        else:
            teamvec2 = frame['is_on_offence']
        #Get x,y,s,o,dir and for current player
        x1 = frame.iloc[i]['x']
        y1 = frame.iloc[i]['y']
        s1 = frame.iloc[i]['s']
        o1 = frame.iloc[i]['o']
        dir1 = frame.iloc[i]['dir']
        # Assign relative values
        temp[0,0:-2,i] = torch.tensor((frame['x'] - x1).values)
        temp[1,0:-2,i] = torch.tensor((frame['y'] - y1).values)
        temp[2,0:-2,i] = torch.tensor((frame['s'] - s1).values)
        temp[3,0:-2,i] = torch.tensor((frame['dir'] - dir1).values)
        temp[3,0:-2,i] = temp[3,0:-2,i] - (temp[3,0:-2,j] > 360)*-360
        temp[3,0:-2,i] = temp[3,0:-2,i] + (temp[3,0:-2,j] < 0)*-360
        temp[4,0:-2,i] = torch.tensor((np.arctan2((frame['y'] - y1).to_numpy(), -1*(frame['x'] - x1).to_numpy())*180/np.pi))
        temp[4,0:-2,i] = temp[4,0:-2,i] + (1*(-1*(frame['x'] - x1) > 0).values & (((frame['y'] - y1) < 0)*180).values) + (-1*(-1*(frame['x'] - x1) > 0).values & ((frame['y'] - y1) < 0).values)*180
        temp[4,0:-2,i] = temp[4,0:-2,i] -360*(temp[4,0:-2,i] > 360)
        temp[4,0:-2,i] = temp[4,0:-2,i] +360*(temp[4,0:-2,i] < 0)
        temp[4,0:-2,i] -= o1
        #This is fine
        temp[5,0:-2,i] = torch.tensor((teamvec2*(((frame['x'] - x1)**2 + (frame['y'] - y1)**2)**0.5) - (~teamvec2)*(((frame['x'] - x1)**2 + (frame['y'] - y1)**2)**0.5)).values)
        # Get distance from sideline and goalline
        if y1 >= 53.3/2:
            temp[:,i] = 53.3 - y1
        else:
            temp[:,i] = y1
                                    
        temp[-1,i] = 105 - x1
    #Sort by player distance from ball carrier, with defensive position being negative
    inds = temp[5,ballcarrier,:].sort()[1]
    temp = temp[:,:,inds]
    temp = temp[:,inds,:]
    return temp

def predict(frame,model):
    #seperate predicted yards and attention mechanism
    yards_predict, att = model.forward(reformat(frame))
    #return predicted yards
    return yards_predict

In [274]:
class AttentionBlock(nn.Module):
    def __init__(self, in_features_l, in_features_g, attn_features, up_factor, normalize_attn=True):
        super(AttentionBlock, self).__init__()
        self.up_factor = up_factor
        self.normalize_attn = normalize_attn
        self.W_l = nn.Conv2d(in_channels=in_features_l, out_channels=attn_features, kernel_size=1, padding=0, bias=False)
        self.W_g = nn.Conv2d(in_channels=in_features_g, out_channels=attn_features, kernel_size=1, padding=0, bias=False)
        self.phi = nn.Conv2d(in_channels=attn_features, out_channels=1, kernel_size=1, padding=0, bias=True)
        self.relu = nn.SiLU()
    def forward(self, l, g):
        #print(in_features_g)
        N, C, W, H = l.shape
        l_ = self.W_l(l)
        g_ = self.W_g(g)
        #print(g_.shape)
        c = self.phi(self.relu(l_ + g_)) # batch_sizex1xWxH
        #print(c.shape)
        # compute attn map
        if self.normalize_attn:
            a = (c.view(N,1,-1)).view(N,1,W,H)
        else:
            a = torch.sigmoid(c)
        # re-weight the local feature
        f = torch.mul(a.expand_as(l), l) # batch_sizexCxWxH
        if self.normalize_attn:
            output = f.view(N,C,-1) # weighted sum
        else:
            output = nn.AdaptiveAvgPool2d(f, (1,1)).view(N,C) # global average pooling
        return a, output
    
class Skynet1_4(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv21 = nn.Conv2d(8,128,1,stride = 1)
        self.conv22 = nn.Conv2d(128, 160, 1,stride = 1)
        self.conv23 = nn.Conv2d(160,128, 1,stride = 1)
        self.poolmax2 = nn.MaxPool2d(1,5)
        self.poolavg2 = nn.AvgPool2d(1,5)

        self.poolmax1 = nn.MaxPool1d(1,5)
        self.poolavg1 = nn.AvgPool1d(1,5)
        self.batch1 = nn.BatchNorm1d(2)

        self.conv11 = nn.Conv1d(2,128,1)
        self.batch2 = nn.BatchNorm1d(128)
        self.conv12 = nn.Conv1d(128,160,1)
        self.batch3 = nn.BatchNorm1d(160)
        self.conv13 = nn.Conv1d(160,96,1)
        self.batch4 = nn.BatchNorm1d(96)

        self.attentionmech = AttentionBlock(8, 128, 256, 4, normalize_attn=True)
        self.fc1 = nn.Linear(1920,512)
        self.fc2 = nn.Linear(512,256)
        self.batch6 = nn.BatchNorm1d(512)
        self.batch5 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256,1)
        #self.squeeze = torch.squeeze(1)
        self.relu = nn.SiLU()

    def forward(self,x):
        temp = x
        x = self.relu(self.conv21(x))
        x = self.relu(self.conv22(x))
        x = self.relu(self.conv23(x))
        attention, x = self.attentionmech(temp,x)
        xmax = self.poolmax2(x)
        xavg = self.poolavg2(x)
        x = 0.3*xmax + 0.7*xavg
        #print(x.shape)
        
        #x = torch.flatten(x,start_dim=2,end_dim=3)
        x = self.batch1(x)
        x = self.relu(self.conv11(x))
        x = self.batch2(x)
        x = self.relu(self.conv12(x))
        x = self.batch3(x)
        x = self.relu(self.conv13(x))
        x = self.batch4(x)
        xmax = self.poolmax1(x)
        xavg = self.poolavg1(x)
        x = 0.3*xmax + 0.7*xavg
        x = torch.flatten(x,start_dim=1,end_dim=2)
        #print(x.shape)

        x = self.relu(self.fc1(x))
        #print(x.mT.shape)
        x = self.batch6(x)
        x = self.relu(self.fc2(x))
        #x = self.batch5(x.T)
        x = self.fc3(x)
        return x, attention

In [266]:
#Load the model, requires load file to be in same directory
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Skynet1_4().to(device).eval()
model.load_state_dict(torch.load("modelSkynet15-4fixed.pt", map_location=device))

<All keys matched successfully>

In [289]:
def generate_counter_factuals(tracking, gameId, playId, time, prev_locations):
    tracking_single=tracking.loc[(tracking['playId'] == playId) & (tracking['time']==time) & (tracking['gameId']==gameId) & (tracking['nflId'].isna()==False)]
    tracking_single=preprocessing(tracking_single)
    tracking_single=tracking_single.merge(prev_locations, how='inner', on=['gameId', 'playId', 'nflId'])
    defensive_ids=tracking_single.loc[tracking_single['is_on_defence']==True]['nflId']
    tracking_single['yards_saved']=0
    print(tracking_single[['x', 'y', 's', 'dir', 'o', 'is_on_offence', 'is_on_defence', 'is_ballcarrier']].iloc[0])
    regular_prediction=predict(tracking_single[['x', 'y', 's', 'dir', 'o', 'is_on_offence', 'is_on_defence', 'is_ballcarrier']], model)
    for i in defensive_ids:
        passed_df=tracking_single.copy()
        list_values=list(passed_df.loc[passed_df['nflId']==i].iloc[0][['PROJ_x', 'PROJ_y', 'OLD_s', 'OLD_dir', 'OLD_o']])
        passed_df.loc[passed_df['nflId']==i, ['x', 'y', 's', 'dir', 'o']]=list_values
        tracking_single.loc[tracking_single['nflId']==i, 'yards_saved']=predict(passed_df[['x', 'y', 's', 'dir', 'o', 'is_on_offence', 'is_on_defence', 'is_ballcarrier']], model)-regular_prediction
    return tracking_single.loc[tracking_single['is_on_defence']==True][['nflId', 'yards_saved']]

In [290]:
def generate_counter_factual_movements(tracking, gameId, playId):
    distinctTimes=tracking.loc[(tracking['playId'] == playId)& (tracking['gameId']==gameId) ]['time'].unique()
    previous_defense_time=distinctTimes[0]
    savedTable={}
    for i in range(1, len(distinctTimes[1:])):
        prev_location=tracking.loc[(tracking['playId'] == playId) & (tracking['time']==previous_defense_time) & (tracking['gameId']==gameId)]
        prev_location=preprocessing(prev_location, True) 
        dfForRunning=generate_counter_factuals(tracking, gameId, playId, distinctTimes[i], prev_location)
        if i==1:
            savedTable=dfForRunning
        else:
            savedTable=savedTable.merge(dfForRunning, how='outer', on='nflId', suffixes=('_table1', '_table2'))
            savedTable['yards_saved']=savedTable['yards_saved_table1']+savedTable['yards_saved_table2']
            savedTable=savedTable[['nflId', 'yards_saved']]
        previous_defense_time=distinctTimes[i]
    return savedTable

In [291]:
warnings.filterwarnings('ignore')

generate_counter_factual_movements(tracking1_with_plays, 2022090800, 56)

x                     31.53
y                 26.203333
s                      1.67
dir                  328.53
o                     50.98
is_on_offence          True
is_on_defence         False
is_ballcarrier        False
Name: 0, dtype: object


RuntimeError: The expanded size of the tensor (20) must match the existing size (22) at non-singleton dimension 0.  Target sizes: [20].  Tensor sizes: [22]

In [ ]:
def processToVisualize(tracking, play, game_info, gameId, playId, time):
  tracking_single=tracking.loc[(tracking['playId'] == playId) & (tracking['time']==time) & (tracking['gameId']==gameId)]
  testingNew=pd.merge(tracking_single, play, on=['gameId', 'playId'], how='inner')
  testingNew=pd.merge(testingNew, game_info, on=['gameId'], how='inner')
  testingNew['radiansDirection'] = testingNew['dir'].astype(float).apply(math.radians) #Converts angle in degrees to radians
  testingNew['xComponent']=testingNew['radiansDirection'].astype(float).apply(math.cos) #Converts angle into an x and y component
  testingNew['yComponent']=testingNew['radiansDirection'].astype(float).apply(math.sin)
  testingNew['xspeed']=testingNew['xComponent']*testingNew['s'] #Determines magnitude of speed by multiplying x and y component by magnitude of speed
  testingNew['yspeed']=testingNew['yComponent']*testingNew['s']
  return testingNew

In [ ]:
def generate_predictions_df